In [133]:
import sympy as sy
import numpy as np
from scipy.optimize import minimize, NonlinearConstraint

In [134]:
gamma1, gamma2, gamma3, s1, s2, lam1, lam2 = sy.symbols('gamma_1, gamma_2, gamma_3, s_1, s_2, lambda_1, lambda_2')
gamma = sy.Matrix([gamma1, gamma2, gamma3])
a11, a12, a13, a21, a22, a23, a31, a32, a33 = sy.symbols('a_11, a_12, a_13, a_21, a_22, a_23, a_31, a_32, a_33')
b1, b2, b3 = sy.symbols('b_1, b_2, b_3')
variables = [gamma1, gamma2, gamma3, s1, s2, lam1, lam2]

In [135]:
A = np.loadtxt('./cpcol/data/A.txt')
b = np.loadtxt('./cpcol/data/b.txt')

A, b

(array([[ 2.00000000e+00, -1.39888101e-16,  0.00000000e+00],
        [-1.39888101e-16,  2.00000000e+00,  0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  2.00000000e+00]]),
 array([-1.2,  1.6,  0. ]))

In [136]:
# A = sy.Matrix([[a11, a12, a13],
#                 [a21, a22, a23],
#                 [a31, a32, a33]])
# b = sy.Matrix([b1, b2, b3])
# mu = sy.symbols('mu')
# A = sy.Matrix([[2, 0, 0], [0, 2, 0], [0, 0, 2]])
# b = sy.Matrix([-2, 0, 0])
# A = sy.Matrix(np.random.rand(3, 3))
# b = sy.Matrix(np.random.rand(3))
# A = np.array([0.9172,    0.7537,    0.0759,
#     0.2858,    0.3804,    0.0540,
#     0.7572,    0.5678,    0.5308]).reshape(3, 3)


# b = np.array([0.7792,
#     0.9340,
#     0.1299])
# A = sy.Matrix(A)
# b = sy.Matrix(b)

mu = 1
A = sy.Matrix(A)
b = sy.Matrix(b)

In [137]:
f = sy.Rational(1, 2)*gamma.T@A@gamma + b.T@gamma
L = f[0] + lam1*(mu**2*gamma1**2 - gamma2**2 - gamma3**2 - s1**2) + lam2*(mu*gamma1 - s2**2)
grad_L = [sy.diff(L, variables[i]) for i in range(len(variables))]
H = sy.Rational(1, 2)*sum([entry**2 for entry in grad_L])
constraint = mu*gamma1 - sy.sqrt(gamma2**2 + gamma3**2) # this whole thing should be >=0
H

2*lambda_1**2*s_1**2 + 2*lambda_2**2*s_2**2 + (gamma_1 - s_2**2)**2/2 + 2*(-gamma_3*lambda_1 + 1.0*gamma_3)**2 + 2.0*(-6.99440505513849e-17*gamma_1 - 1.0*gamma_2*lambda_1 + gamma_2 + 0.8)**2 + (gamma_1**2 - gamma_2**2 - gamma_3**2 - s_1**2)**2/2 + 2*(gamma_1*lambda_1 + 1.0*gamma_1 - 6.99440505513849e-17*gamma_2 + lambda_2/2 - 0.6)**2

In [138]:
X1, X2, X3, X4, X5, X6, X7 = sy.symbols('X1, X2, X3, X4, X5, X6, X7')
q_variables = [X1, X2, X3, X4, X5, X6, X7]
replacements = [(gamma1, X1), (gamma2, X2), (gamma3, X3), (s1, X4), (s2, X5), (lam1, X6), (lam2, X7)]
H_new = H.subs(replacements)
H_new

2*X4**2*X6**2 + 2*X5**2*X7**2 + (X1 - X5**2)**2/2 + 2*(-X3*X6 + 1.0*X3)**2 + 2.0*(-6.99440505513849e-17*X1 - 1.0*X2*X6 + X2 + 0.8)**2 + (X1**2 - X2**2 - X3**2 - X4**2)**2/2 + 2*(X1*X6 + 1.0*X1 - 6.99440505513849e-17*X2 + X7/2 - 0.6)**2

In [139]:
Jacobian = sy.Matrix([sy.diff(H_new, q_variables[i]) for i in range(len(q_variables))]).T
Jacobian

Matrix([[2*X1*(X1**2 - X2**2 - X3**2 - X4**2) + 1.0*X1 + 2.79776202205539e-16*X2*X6 - 2.7977620220554e-16*X2 - X5**2 + 2*(2*X6 + 2.0)*(X1*X6 + 1.0*X1 - 6.99440505513849e-17*X2 + X7/2 - 0.6) - 2.23820961764432e-16, -2.79776202205539e-16*X1*X6 - 2.79776202205539e-16*X1 - 2*X2*(X1**2 - X2**2 - X3**2 - X4**2) + 1.95686808301387e-32*X2 - 1.3988810110277e-16*X7 + 2.0*(2 - 2.0*X6)*(-6.99440505513849e-17*X1 - 1.0*X2*X6 + X2 + 0.8) + 1.67865721323324e-16, -2*X3*(X1**2 - X2**2 - X3**2 - X4**2) + 2*(2.0 - 2*X6)*(-X3*X6 + 1.0*X3), 4*X4*X6**2 - 2*X4*(X1**2 - X2**2 - X3**2 - X4**2), 4*X5*X7**2 - 2*X5*(X1 - X5**2), 4*X1*(X1*X6 + 1.0*X1 - 6.99440505513849e-17*X2 + X7/2 - 0.6) - 4.0*X2*(-6.99440505513849e-17*X1 - 1.0*X2*X6 + X2 + 0.8) - 4*X3*(-X3*X6 + 1.0*X3) + 4*X4**2*X6, 2*X1*X6 + 2.0*X1 - 1.3988810110277e-16*X2 + 4*X5**2*X7 + X7 - 1.2]])

In [140]:
H_fun = sy.lambdify([(X1, X2, X3, X4, X5, X6, X7)], H_new)
f_fun = sy.lambdify([(gamma1, gamma2, gamma3)], f)
constraint_fun = sy.lambdify([(gamma1, gamma2, gamma3)], constraint)

def jac(x):
    Jacobian_fun = sy.lambdify([(X1, X2, X3, X4, X5, X6, X7)], Jacobian)
    return Jacobian_fun(x).reshape(7)
H_fun(np.zeros(7)), jac(np.ones(7)), f_fun(np.zeros(3))

(np.float64(2.0),
 array([11.2,  4. ,  4. ,  8. ,  4. ,  8.4,  7.8]),
 array([[0.]]))

In [141]:
gamma_n0 = np.random.uniform(0, 1)
scale = np.random.uniform(-1, 1, 2)
gamma0 = np.array([gamma_n0, scale[0]*np.sqrt(gamma_n0**2/2), scale[0]*np.sqrt(gamma_n0**2/2)])
s0 = np.random.uniform(0, 1, 2)
lam0 = -np.random.uniform(0, 1, 2)
x0 = np.concatenate((gamma0, s0, lam0))
sol = minimize(H_fun, x0, method='BFGS', jac=jac)
sol

  message: Optimization terminated successfully.
  success: True
   status: 0
      fun: 5.332362426314192e-12
        x: [ 7.000e-01 -7.000e-01 -3.947e-07  7.713e-06  8.367e-01
            -1.429e-01 -5.160e-07]
      nit: 17
      jac: [ 1.373e-06  4.998e-06 -2.062e-06  6.296e-07 -1.301e-06
             2.211e-06 -1.643e-06]
 hess_inv: [[ 3.273e-01 -2.681e-02 ... -1.410e-01 -9.524e-02]
            [-2.681e-02  2.362e-01 ... -2.200e-01  9.404e-02]
            ...
            [-1.410e-01 -2.200e-01 ...  5.725e-01 -1.487e-01]
            [-9.524e-02  9.404e-02 ... -1.487e-01  3.574e-01]]
     nfev: 20
     njev: 20

In [142]:
A = np.array(A)
b = np.array(b)
gam = sol.x[:3]
f_fun(gam), constraint_fun(sol.x[:3])
true = minimize(f_fun, x0[:3], constraints=NonlinearConstraint(constraint_fun, 0, np.inf))
true

 message: Optimization terminated successfully
 success: True
  status: 0
     fun: -0.9800001768382682
       x: [ 7.001e-01 -7.001e-01 -4.856e-05]
     nit: 6
     jac: [ 2.001e-01  1.999e-01 -9.711e-05]
    nfev: 26
    njev: 6

In [143]:
print(sol.x[:3], '\n', true.x, '\n\nError:', np.linalg.norm(sol.x[:3] - true.x))

[ 6.99999908e-01 -6.99999431e-01 -3.94695439e-07] 
 [ 7.00063336e-01 -7.00064272e-01 -4.85626164e-05] 

Error: 0.0001027021596859936
